In [3]:
%pip install fastapi uvicorn nest_asyncio torch

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pytorch-cpu (from versions: none)
ERROR: No matching distribution found for pytorch-cpu


In [5]:
from fastapi import FastAPI
from pydantic import BaseModel

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.quantization import quantize_dynamic

import nest_asyncio
import uvicorn

# Patch asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = AutoModelForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-2.7B")

# Apply dynamic quantization to the model
model_quantized = quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_quantized.to(device)

class TextInput(BaseModel):
    text: str

@app.post("/inference/")
async def perform_inference(input: TextInput):
    #inputs = tokenizer(text, return_tensors="pt")
    inputs = tokenizer(input.text, return_tensors="pt").to(device)
    with torch.no_grad():
        #outputs = model(**inputs)
        output = model_quantized(**inputs)
    # Process the model outputs as needed
    return {"result": outputs.logits.argmax().item()}
#    return {"message": "Inference successful", "text": input.text}


OSError: your-model-name is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`